## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# export
from exp.nb_01 import *


def get_data():
    """Download MNIST dataset and return training and validation tensors."""
    path = datasets.download_data(MNIST_URL, ext=".gz")

    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

    return map(tensor, (x_train, y_train, x_valid, y_valid))


def normalize(x, mean, std):
    """Normalize x by first centering it and then scale it."""
    return (x - mean) / std

<IPython.core.display.Javascript object>

In [3]:
# Get MNIST data
x_train, y_train, x_valid, y_valid = get_data()
x_train.shape, x_valid.shape

(torch.Size([50000, 784]), torch.Size([10000, 784]))

<IPython.core.display.Javascript object>

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

<IPython.core.display.Javascript object>

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

<IPython.core.display.Javascript object>

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.0001), tensor(1.))

<IPython.core.display.Javascript object>

Now the images have a mean of 0 and a standard deviation of 1.

In [7]:
# export
def test_near_zero(a, tol=1e-3):
    assert a.abs() < tol, f"Near zero: {a}"

<IPython.core.display.Javascript object>

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1 - x_train.std())

<IPython.core.display.Javascript object>

In [9]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

<IPython.core.display.Javascript object>

We have 10 classes.

## Foundations version

### Basic architecture

In [10]:
# num hidden
nh = 50

<IPython.core.display.Javascript object>

Dividing the initialization by the square root of the number of hidden units in each layer will result in weight matrix that has mean of ~ 0 and std ~ 1 / sqrt(# hidden units).

In [11]:
# simplified kaiming init / he init
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) / math.sqrt(nh)
b2 = torch.zeros(1)

<IPython.core.display.Javascript object>

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1 / math.sqrt(m))

<IPython.core.display.Javascript object>

In [13]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(), x_valid.std()

(tensor(-0.0057), tensor(0.9924))

<IPython.core.display.Javascript object>

In [14]:
def lin(x, w, b):
    return x @ w + b

<IPython.core.display.Javascript object>

In [15]:
t = lin(x_valid, w1, b1)

<IPython.core.display.Javascript object>

In [16]:
# ...so should this, because we used kaiming init, which is designed to do this
t.mean(), t.std()

(tensor(-0.1182), tensor(0.9789))

<IPython.core.display.Javascript object>

In [17]:
def relu(x):
    return x.clamp_min(0.0)

<IPython.core.display.Javascript object>

In [18]:
t = relu(lin(x_valid, w1, b1))

<IPython.core.display.Javascript object>

In [19]:
# ...actually it really should be this!
t.mean(), t.std()

(tensor(0.3299), tensor(0.5443))

<IPython.core.display.Javascript object>

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [20]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh) * math.sqrt(2 / m)

<IPython.core.display.Javascript object>

In [21]:
w1.mean(), w1.std()

(tensor(-0.0003), tensor(0.0504))

<IPython.core.display.Javascript object>

In [22]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.6183), tensor(0.8452))

<IPython.core.display.Javascript object>

In [23]:
# export
from torch.nn import init

<IPython.core.display.Javascript object>

- If we use `mode=fan_in`, this means we preserve the variance of the forward pass by dividing by sqrt(1 / m); where `m` is the number of hidden units in the input layer.
- If we use `mode=fan_out`, this means we preserve the variance of the backward pass by dividing by sqrt(1 / nh); where `nh` is the number of hidden units in the output layer.

In [24]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode="fan_out")
t = relu(lin(x_valid, w1, b1))

<IPython.core.display.Javascript object>

We are using `fan_out` instead of `fan_in` because PyTorch returns the weight nh x m since the linear layer operation is `x.matmul(w.t())`.

In [25]:
w1.mean(), w1.std()

(tensor(-0.0004), tensor(0.0504))

<IPython.core.display.Javascript object>

In [26]:
init.kaiming_normal_??

<IPython.core.display.Javascript object>

Signature: init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where

    .. math::
        \text{std} = \frac{\text{gain}}{\sqrt{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``

In [27]:
torch.nn.init._calculate_correct_fan??

<IPython.core.display.Javascript object>

Signature: torch.nn.init._calculate_correct_fan(tensor, mode)
Docstring: <no docstring>
Source:   
def _calculate_correct_fan(tensor, mode):
    mode = mode.lower()
    valid_modes = ['fan_in', 'fan_out']
    if mode not in valid_modes:
        raise ValueError("Mode {} not supported, please use one of {}".format(mode, valid_modes))

    fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
    return fan_in if mode == 'fan_in' else fan_out
File:      ~/anaconda3/envs/dl-zoo/lib/python3.7/site-packages/torch/nn/init.py
Type:      function


In [28]:
torch.nn.Linear??

<IPython.core.display.Javascript object>

Init signature: torch.nn.Linear(in_features: int, out_features: int, bias: bool = True) -> None
Source:        
class Linear(Module):
    r"""Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to ``False``, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(N, *, H_{in})` where :math:`*` means any number of
          additional dimensions and :math:`H_{in} = \text{in\_features}`
        - Output: :math:`(N, *, H_{out})` where all but the last dimension
          are the same shape as the input and :math:`H_{out} = \text{out\_features}`.

    Attributes:
        weight: the learnable weights of the module of shape
            :math:`(\text{out\_features}, \text{in\_features})`. The values are
            initialized from :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})`, where
           

In [29]:
torch.nn.init._calculate_fan_in_and_fan_out??

<IPython.core.display.Javascript object>

Signature: torch.nn.init._calculate_fan_in_and_fan_out(tensor)
Docstring: <no docstring>
Source:   
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.dim()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with fewer than 2 dimensions")

    num_input_fmaps = tensor.size(1)
    num_output_fmaps = tensor.size(0)
    receptive_field_size = 1
    if tensor.dim() > 2:
        receptive_field_size = tensor[0][0].numel()
    fan_in = num_input_fmaps * receptive_field_size
    fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out
File:      ~/anaconda3/envs/dl-zoo/lib/python3.7/site-packages/torch/nn/init.py
Type:      function


In [30]:
t.mean(), t.std()

(tensor(0.5288), tensor(0.7931))

<IPython.core.display.Javascript object>

In [31]:
w1.shape

torch.Size([784, 50])

<IPython.core.display.Javascript object>

In [32]:
import torch.nn

<IPython.core.display.Javascript object>

In [33]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

<IPython.core.display.Javascript object>

In [34]:
torch.nn.Linear.forward??

<IPython.core.display.Javascript object>

Signature: torch.nn.Linear.forward(self, input: torch.Tensor) -> torch.Tensor
Docstring: <no docstring>
Source:   
    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)
File:      ~/anaconda3/envs/dl-zoo/lib/python3.7/site-packages/torch/nn/modules/linear.py
Type:      function


In [35]:
torch.nn.functional.linear??
# input.matmul(weight.t())

<IPython.core.display.Javascript object>

Signature: torch.nn.functional.linear(input, weight, bias=None)
Source:   
def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

    Shape:

        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    tens_ops = (input, weight)
    if not torch.jit.is_scripting():
        if any([type(t) is not Tensor for t in tens_ops]) and has_torch_function(tens_ops):
            return handle_torch_function(linear, tens_ops, input, weight, bias=bias)
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())
        if bias is not None:
     

In [36]:
torch.nn.Conv2d??

<IPython.core.display.Javascript object>

Init signature:
torch.nn.Conv2d(
    in_channels: int,
    out_channels: int,
    kernel_size: Union[int, Tuple[int, int]],
    stride: Union[int, Tuple[int, int]] = 1,
    padding: Union[int, Tuple[int, int]] = 0,
    dilation: Union[int, Tuple[int, int]] = 1,
    groups: int = 1,
    bias: bool = True,
    padding_mode: str = 'zeros',
)
Source:        
class Conv2d(_ConvNd):
    r"""Applies a 2D convolution over an input signal composed of several input
    planes.

    In the simplest case, the output value of the layer with input size
    :math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
    can be precisely described as:

    .. math::
        \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
        \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


    where :math:`\star` is the valid 2D `cross-correlation`_ operator,
    :math:`N` is a batch size, :math:`C` deno

In [37]:
torch.nn.modules.conv._ConvNd.reset_parameters??

<IPython.core.display.Javascript object>

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self) -> None
Docstring: <no docstring>
Source:   
    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/envs/dl-zoo/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


This is how PyTorch initialized the Conv layer:
- `init.kaiming_uniform_(self.weight, a=math.sqrt(5))`

It is not documented where the sqrt(5) comes from.

The following **RELU** is based on Jeremy's experimentation and he said it works better.

In [38]:
# what if...?
def relu(x):
    return x.clamp_min(0.0) - 0.5

<IPython.core.display.Javascript object>

In [39]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh) * math.sqrt(2.0 / m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0393), tensor(0.8152))

<IPython.core.display.Javascript object>

In [40]:
def model(xb):
    """One hidden layer model."""
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

<IPython.core.display.Javascript object>

In [41]:
%timeit -n 10 _ = model(x_valid)

4.99 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<IPython.core.display.Javascript object>

In [42]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

<IPython.core.display.Javascript object>

### Loss function: MSE

In [43]:
model(x_valid).shape

torch.Size([10000, 1])

<IPython.core.display.Javascript object>

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [44]:
# export
def mse(output, targ):
    """Compute Mean Squared Error."""
    return (output.squeeze(-1) - targ).pow(2).mean()

<IPython.core.display.Javascript object>

In [45]:
y_train, y_valid = y_train.float(), y_valid.float()

<IPython.core.display.Javascript object>

In [46]:
preds = model(x_train)

<IPython.core.display.Javascript object>

In [47]:
preds.shape

torch.Size([50000, 1])

<IPython.core.display.Javascript object>

In [48]:
mse(preds, y_train)

tensor(29.6917)

<IPython.core.display.Javascript object>

### Gradients and backward pass

In [49]:
def mse_grad(y_hat, targ):
    # grad of loss with respect to output of previous layer
    y_hat.g = 2.0 * (y_hat.squeeze() - targ).unsqueeze(-1) / y_hat.shape[0]

<IPython.core.display.Javascript object>

In [50]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp > 0).float() * out.g

<IPython.core.display.Javascript object>

In [51]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)
    assert w.shape == w.g.shape
    assert b.shape == b.g.shape
    inp.g = out.g @ w.t()

<IPython.core.display.Javascript object>

In [52]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)

    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

<IPython.core.display.Javascript object>

In [53]:
forward_and_backward(x_train, y_train)

<IPython.core.display.Javascript object>

In [54]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

<IPython.core.display.Javascript object>

We cheat a little bit and use PyTorch autograd to check our results.

In [55]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

<IPython.core.display.Javascript object>

In [56]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

<IPython.core.display.Javascript object>

In [57]:
loss = forward(xt2, y_train)

<IPython.core.display.Javascript object>

In [58]:
loss.backward()

<IPython.core.display.Javascript object>

In [59]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)

<IPython.core.display.Javascript object>

Now we are sure that our implementation of both forward and backward passes are correct after comparing it with PyTorch.

## Refactor model

### Layers as classes

In [60]:
class Relu:
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.0) - 0.5
        return self.out

    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

<IPython.core.display.Javascript object>

In [61]:
class Lin:
    def __init__(self, w, b):
        self.w, self.b = w, b

    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out

    def backward(self):
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)
        self.inp.g = self.out.g @ self.w.t()

<IPython.core.display.Javascript object>

In [62]:
class Mse:
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (self.inp.squeeze() - self.targ).pow(2).mean()
        return self.out

    def backward(self):
        self.inp.g = (
            2.0 * (self.inp.squeeze(-1) - self.targ).unsqueeze(-1) / self.targ.shape[0]
        )

<IPython.core.display.Javascript object>

In [63]:
class Model:
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()

    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)

    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

<IPython.core.display.Javascript object>

In [64]:
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model(w1, b1, w2, b2)

<IPython.core.display.Javascript object>

In [65]:
%time loss = model(x_train, y_train)

CPU times: user 101 ms, sys: 1.41 ms, total: 103 ms
Wall time: 25.6 ms


<IPython.core.display.Javascript object>

In [66]:
%time model.backward()

CPU times: user 289 ms, sys: 164 ms, total: 453 ms
Wall time: 117 ms


<IPython.core.display.Javascript object>

In [67]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

<IPython.core.display.Javascript object>

### Module.forward()

In [68]:
class Module:
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self):
        raise Exception("not implemented")

    def backward(self):
        self.bwd(self.out, *self.args)

<IPython.core.display.Javascript object>

In [69]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.0) - 0.5

    def bwd(self, out, inp):
        inp.g = (inp > 0).float() * out.g

<IPython.core.display.Javascript object>

In [70]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b

    def forward(self, inp):
        return inp @ self.w + self.b

    def bwd(self, out, inp):
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)
        inp.g = out.g @ self.w.t()

<IPython.core.display.Javascript object>

In [71]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()

    def bwd(self, out, inp, targ):
        inp.g = 2 * (inp.squeeze() - targ).unsqueeze(1) / targ.shape[0]

<IPython.core.display.Javascript object>

In [72]:
class Model:
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()

    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)

    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

<IPython.core.display.Javascript object>

In [73]:
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model()

<IPython.core.display.Javascript object>

In [74]:
%time loss = model(x_train, y_train)

CPU times: user 103 ms, sys: 1.33 ms, total: 104 ms
Wall time: 26 ms


<IPython.core.display.Javascript object>

In [75]:
%time model.backward()

CPU times: user 272 ms, sys: 158 ms, total: 430 ms
Wall time: 111 ms


<IPython.core.display.Javascript object>

In [76]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

<IPython.core.display.Javascript object>

### nn.Linear and nn.Module

In [77]:
# export
from torch import nn

<IPython.core.display.Javascript object>

In [78]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse

    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x.squeeze(), targ)

<IPython.core.display.Javascript object>

In [79]:
model = Model(m, nh, 1)

<IPython.core.display.Javascript object>

In [80]:
%time loss = model(x_train, y_train)

CPU times: user 92 ms, sys: 1.35 ms, total: 93.4 ms
Wall time: 24.4 ms


<IPython.core.display.Javascript object>

In [81]:
%time loss.backward()

CPU times: user 113 ms, sys: 3.16 ms, total: 116 ms
Wall time: 28.9 ms


<IPython.core.display.Javascript object>

## Export

In [82]:
!python ../src/notebook2script.py 02_Fully-Connected.ipynb

Converted 02_Fully-Connected.ipynb to exp/nb_02.py


<IPython.core.display.Javascript object>